In [ ]:
from ultralytics import YOLO

In [ ]:
# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

In [ ]:
# Train the model on the COCO8 dataset for 100 epochs
train_results = model.train(
    data="coco8.yaml",  # Path to dataset configuration file
    epochs=100,  # Number of training epochs
    imgsz=640,  # Image size for training
    device="cpu",  # Device to run on (e.g., 'cpu', 0, [0,1,2,3])
)

In [ ]:
# Evaluate the model's performance on the validation set
metrics = model.val()

In [ ]:
!ls /home/shravan/documents/deeplearning/gen_ai/genai_lab/cv-tutorials/datasets/coco8/

In [ ]:
from pathlib import Path
import cv2
from matplotlib import pyplot as plt
%matplotlib inline


In [ ]:
from ultralytics import YOLO
import os

# Load model
# model = YOLO("yolov8n.pt")  # or your custom .pt model

val_dir = Path("/home/shravan/documents/deeplearning/gen_ai/genai_lab/cv-tutorials/datasets/coco8/images/val")
image_paths = sorted(val_dir.glob("*.jpg"))

for img_path in image_paths:
    results = model(str(img_path))
    
    # Get image with boxes
    pred_img = results[0].plot()
    pred_img_rgb = cv2.cvtColor(pred_img, cv2.COLOR_BGR2RGB)

    # Display in notebook
    plt.imshow(pred_img_rgb)
    plt.title(f"Predictions: {img_path.name}")
    plt.axis("off")
    plt.show()


In [ ]:
# Define remote image or video URL
source = "https://ultralytics.com/images/bus.jpg"
source = "https://www.team-bhp.com/forum/attachments/super-cars-imports-india/2037462d1596381376-111-garage-tasteful-car-collection-hyderabad-mustang-mach-1.jpg"

# Run inference on the source
results = model(source)  # list of Results objects

In [ ]:
results

In [ ]:
import matplotlib.pyplot as plt

# Get plotted result
im = results[0].plot()  # returns BGR image with boxes drawn

# Convert to RGB and show
plt.imshow(im[..., ::-1])
plt.axis('off')
plt.title("Predictions")
plt.show()


In [ ]:
from ultralytics import YOLO
model = YOLO("yolo11n.pt")
model.export(format="onnx")


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Rerun inference or continue from earlier results
for result in results:  # each frame/image
    img = result.orig_img.copy()  # original BGR image
    boxes = result.boxes  # boxes.xyxy, boxes.cls, boxes.conf

    if boxes is None or boxes.xyxy.shape[0] == 0:
        print("No detections.")
        continue

    for i in range(len(boxes)):
        box = boxes.xyxy[i].cpu().numpy().astype(int)  # [x1, y1, x2, y2]
        cls = int(boxes.cls[i].cpu().item())
        conf = float(boxes.conf[i].cpu().item())
        label = result.names[cls]  # class name

        # Crop box from image
        x1, y1, x2, y2 = box
        cropped = img[y1:y2, x1:x2]  # BGR crop

        # Plot using matplotlib
        plt.figure(figsize=(4, 4))
        plt.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
        plt.title(f"{label} ({conf:.2f})")
        plt.axis("off")
        plt.show()

        # Optional: Wait for key press if looping through video
        input("Press Enter to view next box...")
